In [ ]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=e77da25b07e7c651555951c9989f9833583c84c365727158812bee75b17ac292
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
# Get dataset
# ! pip install datasets
# ! pip install pandas
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("trec", split="train")

# Import into a pandas dataframe, take only the first 1000 rows
df = pd.DataFrame(dataset)[:1000]

# Preview the data to ensure it has loaded correctly
print(df.head(10))

                                                text  coarse_label  fine_label
0  How did serfdom develop in and then leave Russ...             2          26
1   What films featured the character Popeye Doyle ?             1           5
2  How can I find a list of celebrities ' real na...             2          26
3  What fowl grabs the spotlight after the Chines...             1           2
4                    What is the full form of .com ?             0           1
5  What contemptible scoundrel stole the cork fro...             3          29
6  What team did baseball 's St. Louis Browns bec...             3          28
7                    What is the oldest profession ?             3          30
8                           What are liver enzymes ?             2          24
9  Name the scar-faced bounty hunter of The Old W...             3          29


In [ ]:
# We'll set up the name of the model we want to use, the API key, and the input type.
# Create and retrieve a Cohere API key from dashboard.cohere.ai/welcome/register
# Paste your API key here. Remember to not share publicly
import cohere
model_name = "multilingual-22-12"
input_type_embed = "search_query"

# Now we'll set up the cohere client.
co = cohere.Client('h7BI3irdxYktGqLEpKjfUwOFerutik4FLfXuooTI')

# Get the embeddings
embeds = co.embed(texts=list(df['text']),
                  model=model_name,
                  input_type=input_type_embed).embeddings

In [ ]:
# Create the search index, pass the size of embedding
from annoy import AnnoyIndex
import numpy as np
search_index = AnnoyIndex(np.array(embeds).shape[1], 'angular')

# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [ ]:
# Choose an example (we'll retrieve others similar to it)
example_id = 92

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,10,
                                                include_distances=True)

# Format and print the text and distances
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'],
                             'distance': similar_item_ids[1]}).drop(example_id)

print(f"Question:'{df.iloc[example_id]['text']}'\nNearest neighbors:")
print(results) # NOTE: Your results might look slightly different to ours.

Question:'What are bear and bull markets ?'
Nearest neighbors:
                                                 texts  distance
614     What animals do you find in the stock market ?  0.449323
601                  What is `` the bear of beers '' ?  0.468908
137                       What are equity securities ?  0.501946
277                               When is Boxing Day ?  0.513035
547               Where can stocks be traded on-line ?  0.513400
307                       What does NASDAQ stand for ?  0.514046
20                    What is the date of Boxing Day ?  0.525150
922  What is the difference between a median and a ...  0.530480
289  How do I find the balance of my social securit...  0.531263


In [ ]:
similar_item_ids[0]

[92, 614, 601, 137, 277, 547, 307, 20, 922, 289]

In [ ]:
df.iloc[similar_item_ids[0]]

,text,coarse_label,fine_label
92,What are bear and bull markets ?,2,24
614,What animals do you find in the stock market ?,1,2
601,What is `` the bear of beers '' ?,1,2
137,What are equity securities ?,2,24
277,When is Boxing Day ?,5,39
547,Where can stocks be traded on-line ?,4,35
307,What does NASDAQ stand for ?,0,1
20,What is the date of Boxing Day ?,5,39
922,What is the difference between a median and a ...,2,25
289,How do I find the balance of my social securit...,2,26
